# Workshop_04
## Immagine di esempio:

<img src= './img01.jpg'>

In base all'immagine precedente sono state considerate **due** tipologie di tetti:
* **Prima Tipologia**: Mansard Roof
* **Seconda Tipologia** : itersezione fra i 2 modelli Intersecting Roof e Mansard Roof
In base a queste due tipologie verranno poi forniti gli esempi grafici dei risultati.

A causa di alcuni dubbi sulle specifiche del progetto ho deciso di implementare diverse versioni che differenziano fra loro in base al tipo di input.
Il dubbio deriva dal tipo di celle che dovevano essere utilizzate per la creazione del tetto, cioè se era possibile specificare delle celle 1D per creare le travi di sostegno interne e celle 2D per i piani di copertura, o se la struttura del tetto di partenza doveva essere "piena" e costruita con celle 3D.

Sono stati perciò definiti due tipi di codice:
* **caso 1** : la funzione principale prende in input vertici e celle 1D,2D.
* **caso 2** : la funzione principale prende in input vertici e celle 3D.

## CASO 1:
### Codice:

Il codice è composto da più funzioni:
* una funzione ausiliaria *planarity_check(verts)* che controlla la planarità di n vertici.
* una funzione ausiliaria *faces_planarity_check(verts,cells)* che dati in input vertici e celle verifica la planarità dei vertici per ogni cella.
* una funzione ausiliaria *remove_1DCells(cells)* che banalmente rimuove le celle composte da due punti utilizzate per creare gli assi di sostegno interni alla struttura.
* una funzione ausiliaria *create_roof(hpc)* che dato in input un hpc value di un tetto restituisce la struttura completa di travi di sostegno e copertura del tetto.
* una funzione principale *ggpl_mansard_intersecting_roof_builder(verts,cells)* che racchiude le precendeti e prende in input una lista di vertici e una lista di celle che identificano la struttura grezza di un tetto.

In [4]:
from larlib import *
from numpy import *

"""ggpl_mansard_intersecting_roof_builder(verts,cells) è una funzione che dato in input 
una lista di verticie una lista di celle che identificano la struttura grezza di un tetto, 
restituisce la struttura 3D dello stesso tetto più rifinita, dotata di travi di sostegno e copertura.

@verts = lista di vertici
@cells = lista di celle 1D per le travi di sostegno interne, 2D per le "facce" del tetto
"""
def ggpl_mansard_intersecting_roof_builder1(verts,cells):
    
#FUNZIONI AUSILIARIE ------------------------------------------------------------------------#

    """planarity_check(verts) è una funzione che preso in input una lista di n vertici
    controlla la loro complanarità, restituendo False se non sono complanari, True altrimenti.
    
    @verts = lista di n vertici
    """
    def planarity_check(verts):
        """in an n-dimensional space (n>=3) a set of k points are cooplanar if and only if 
        the matrix of their relative differences is of rank 2 or less
        """
        #meno di 4 punti = coplanare
        if(len(verts)>3):
            lastPoint = verts[-1]
            checkMatrix = verts[0:-1]
            for i in range(len(checkMatrix)):
                #ad ogni punto rimasto sottraggo l'ultimo punto
                checkMatrix[i] = np.subtract(checkMatrix[i],lastPoint)
            if(linalg.matrix_rank(matrix(checkMatrix))>2): #controllo rank
                return False
        return True

    """faces_planarity_check(verts,cells) è una funzione che preso in input una lista di n vertici
     e una lista di n celle, controlla la complanarità dei punti per ogni cella, 
     restituendo False se non sono complanari, True altrimenti.
    
    @verts = lista di n vertici
    @cells lista di celle
    """
    def faces_planarity_check(verts, cells):
        for cell in cells:
            vertsToCheck = []
            for i in cell:
                 vertsToCheck += [verts[i-1]] 
            if(not planarity_check(vertsToCheck)):
                return False
        return True
    
    """remove_1DCell(cells) è una funzione che preso in input una lista di n celle
    rimuove le celle 1D che in principio identificavano le travi interne del tetto
    
    @cells = lista di n celle
    """
    def remove_1DCell(cells):
        newCells = []
        for cell in cells:
            if len(cell)>2:
                newCells += [cell]
        return newCells
    
    """create_roof(hpc) è una funzione che preso in input un valore hpc di un tetto grezzo
    ne estrae vertici e celle tramite la funzione UKPOL
    ne verifica la complanarità tramite la funzione faces_planarity_check
    rimuove le celle 1D delle travi di sostegno interne tramite la funzione remove_1DCells
    restituisce una struttura composta dallo scheletro delle travi e la copertura del tetto
    
    @hpc = hpc value della struttura di un tetto grezzo
    """
    def create_roof(hpc):
        ukInfo = UKPOL(SKEL_2(hpc))
        
        if(not faces_planarity_check(ukInfo[0],ukInfo[1])): #ckeck planarity tramite f. ausiliaria
            return None

        beams = STRUCT([OFFSET([0.2,0.2,0.2])(SKEL_1(hpc))]) #struttura delle travi
        verts = list([round(x,3),round(y,3), round(z,3)] for [x,y,z] in ukInfo[0])

        cells = ukInfo[1]
        newCells = remove_1DCell(cells) #rimuovo cell 1D tramite f. ausiliaria

        faces = STRUCT([OFFSET([0.2,0.2,0.2]),(MKPOL([verts, newCells, None]))])

        return STRUCT([T(3)(10),beams,T([3])([0.2]),COLOR(RED)(faces)])

#FINE FUNZIONI AUSILIARIE ----------------------------------------------------------------#
    
    startRoof = (MKPOL([verts,cells,None])) #tetto di partenza
    return create_roof(startRoof)


### Esempio prima tipologia: Mansard Roof

In [5]:
#TETTO DI PARTENZA---------------------------------------------------------------------------#
verts = [[7.,5.,5.],[7.,-5.,5.],[-7.,5.,5.],[-7.,-5.,5.], #rect interno
          [14.,10.,0],[14.,-10.,0],[-14.,10.,0],[-14.,-10.,0], #rect esterno
          [0,0,8.], #punta centrale
          [0,5.,5.],[0,10.,5.],[0,16.,5.],#punta prolungamento
          [-7.,10.,0],[7.,10.,0],[-7.,16.,0],[7.,16.,0], #base prolungamento
          [7.,5.,0],[7.,-5.,0],[-7.,5.,0],[-7.,-5.,0] #punti con z=0 per travi basse
        ]  

cells = [[1,2,5,6],[1,3,5,7],[3,4,7,8],[2,4,6,8], #connessioni rect interna-esterna
          [1,2,9],[1,3,9],[3,4,9],[2,4,9], #connessioni rect interna-puntacentrale
          [17,18],[17,19],[18,20],[19,20],[1,17],[2,18],[4,20],[3,19],[5,17],[6,18],[8,20],[7,19]
        ] 

VIEW(ggpl_mansard_intersecting_roof_builder1(verts,cells))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x0000000004AFED80> >

<img src = './caso1tipo1.PNG'>

### Esempio seconda tipologia: Mansard + Intersect Roof

In [10]:
#TETTO DI PARTENZA---------------------------------------------------------------------------#
verts = [[7.,5.,5.],[7.,-5.,5.],[-7.,5.,5.],[-7.,-5.,5.], #rect interno
          [14.,10.,0],[14.,-10.,0],[-14.,10.,0],[-14.,-10.,0], #rect esterno
          [0,0,8.], #punta centrale
          [0,5.,5.],[0,10.,5.],[0,16.,5.],#punta prolungamento
          [-7.,10.,0],[7.,10.,0],[-7.,16.,0],[7.,16.,0], #base prolungamento
          [7.,5.,0],[7.,-5.,0],[-7.,5.,0],[-7.,-5.,0] #punti con z=0 per travi basse
        ]  

cells = [[1,2,5,6],[1,3,5,7],[3,4,7,8],[2,4,6,8], #connessioni rect interna-esterna
          [1,2,9],[1,3,9],[3,4,9],[2,4,9], #connessioni rect interna-puntacentrale
          [10,11,13],[10,11,14],[11,12,15,13],[11,12,16,14],[12,15,16], #connessioni prolungamento
          [17,18],[17,19],[18,20],[19,20],[1,17],[2,18],[4,20],[3,19],[5,17],[6,18],[8,20],[7,19]
        ] 

VIEW(ggpl_mansard_intersecting_roof_builder1(verts,cells))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x0000000004EAF450> >

<img src = './caso1tipo2.PNG'>

In [11]:
#TETTO DI PARTENZA---------------------------------------------------------------------------#
verts = [[5.,5.,5.],[5.,-5.,5.],[-5.,5.,5.],[-5.,-5.,5.], #rect interno
          [7.,10.,0],[7.,-10.,0],[-7.,10.,0],[-7.,-10.,0], #rect esterno
          [0,0,15.], #punta centrale
          [0,5.,5.],[0,10.,5.],[0,10.,5.],#punta prolungamento
          [-5.,10.,0],[5.,10.,0],[-5.,16.,0],[5.,16.,0], #base prolungamento
          [5.,5.,0],[5.,-5.,0],[-5.,5.,0],[-5.,-5.,0] #punti con z=0 per travi basse
        ]  

cells = [[1,2,5,6],[1,3,5,7],[3,4,7,8],[2,4,6,8], #connessioni rect interna-esterna
          [1,2,9],[1,3,9],[3,4,9],[2,4,9], #connessioni rect interna-puntacentrale
          [10,11,13],[10,11,14],[11,12,15,13],[11,12,16,14],[12,15,16], #connessioni prolungamento
          [17,18],[17,19],[18,20],[19,20],[1,17],[2,18],[4,20],[3,19],[5,17],[6,18],[8,20],[7,19]
        ] 

startRoof = (MKPOL([verts,cells,None])) #tetto di partenza

VIEW(ggpl_mansard_intersecting_roof_builder1(startRoof))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x0000000004EAF360> >

<img src = './caso1tipoplus.PNG'>

## CASO 2 
### Codice:

Il codice è composto da più funzioni:
* una funzione ausiliaria *planarity_check(verts)* che controlla la planarità di n vertici.
* una funzione ausiliaria *faces_planarity_check(verts, cells)* che dati in input vertici e celle verifica la planarità dei vertici per ogni cella.
* una funzione ausiliaria *get_new_verts(verts)* che rimuove quei vertici che non darebbero alcun contributo utile per la costruzione della copertura del tetto, ossia quei vertici "interni" che risultano essere la proiezione su z=0 di altri vertici con z!=0.
* una funzione ausiliaria *remove_base_cells(verts, cells)* che rimuove le celle che identificano dei piani orizzontali interni che non fanno parte della copertura del tetto. [NB: necessario in particolare nell'esempio utilizzato per rimuovere il piano interno che si creava con la creazione del tetto mansardato.] 
* una funzione ausiliaria *create_roof(hpc)* che dato in input un hpc value di un tetto restituisce la struttura completa di travi di sostegno e copertura del tetto.
* una funzione principale *ggpl_mansard_intersecting_roof_builder2(verts,cells)* che racchiude le precendeti e prende in input una lista di vertici e una lista di celle che identificano la struttura grezza di un tetto.

In [6]:
#versione con hpc
from larlib import *
from numpy import *
import itertools

"""ggpl_mansard_intersecting_roof_builder2(verts,cells) è una funzione che dato in input 
una lista di verticie una lista di celle che identificano la struttura grezza di un tetto, 
restituisce la struttura 3D dello stesso tetto più rifinita, dotata di travi di sostegno e copertura.

@verts = lista di vertici
@cells = lista di celle 3D
"""
def ggpl_mansard_intersecting_roof_builder2(verts,cells):
    
#FUNZIONI AUSILIARIE ------------------------------------------------------------------------#

    """planarity_check(verts) è una funzione che preso in input una lista di n vertici
    controlla la loro complanarità, restituendo False se non sono complanari, True altrimenti.
    
    @verts = lista di n vertici
    """
    def planarity_check(verts):
        """in an n-dimensional space (n>=3) a set of k points are cooplanar if and only if 
        the matrix of their relative differences is of rank 2 or less
        """
        #meno di 4 punti = coplanare
        if(len(verts)>3):
            lastPoint = verts[-1]
            checkMatrix = verts[0:-1]
            for i in range(len(checkMatrix)):
                #ad ogni punto rimasto sottraggo l'ultimo punto
                checkMatrix[i] = np.subtract(checkMatrix[i],lastPoint)
            if(linalg.matrix_rank(matrix(checkMatrix))>2): #controllo rank
                return False
        return True

    
    """faces_planarity_check(verts,cells) è una funzione che preso in input una lista di n vertici
     e una lista di n celle, controlla la complanarità dei punti per ogni cella, 
     restituendo False se non sono complanari, True altrimenti.
    
    @verts = lista di n vertici
    @cells lista di celle
    """
    def faces_planarity_check(verts, cells):
        for cell in cells:
            vertsToCheck = []
            for i in cell:
                 vertsToCheck += [verts[i-1]] 
            if(not planarity_check(vertsToCheck)):
                return False
        return True
    
    """get_new_verts() è una funzione che data una lista di vertici che contiene dei punti con z != 0
    che hanno una proiezione sul piano XY con z=0, rimpiazza quei punti con il corrispettivo che ha z != 0
    
    @verts = lista vertici
    """
    def get_new_verts(verts):
        for vert in verts:
            if(vert[2]==0):
                match = [v for v in verts if(vert[0]==v[0] and vert[1] == v[1] and v[2]>0)]
                if(len(match)>0):
                    verts[verts.index(vert)] = match[0]
        return verts
    
    """remove_base_cells è una funzione che data in input una lista di vertici e una lista di celle
    restituisce una lista di celle come quella originale priva di quelle celle che identificano un 
    piano parallelo a XY
    
    @verts = lista vertici
    @cells = lista celle
    """
    def remove_base_cells(verts,cells):
        newCells = []
        for cell in cells:
            z = []
            for i in cell:
                z += [verts[i-1][2]]
            z = list(x for x,_ in itertools.groupby(z))
            if (len(z) > 1):
                newCells += [cell]
        return newCells
    
    """create_roof(hpc) è una funzione che preso in input un valore hpc di un tetto grezzo
    ne estrae vertici e celle tramite la funzione UKPOL
    ne verifica la complanarità tramite la funzione faces_planarity_check
    rimuove vertici e celle non necessarie per la copertura del tetto tramite le funzioni
    get_new_verts e remove_base_cells
    restituisce una struttura composta dallo scheletro delle travi e la copertura del tetto
    
    @hpc = hpc value della struttura di un tetto grezzo
    """ 
    def create_roof(hpc):
        ukInfo = UKPOL(SKEL_2(hpc))
        if(not faces_planarity_check(ukInfo[0],ukInfo[1])): #ckeck planarity tramite f. ausiliaria
            return None

        beams = STRUCT([OFFSET([0.2,0.2,0.2])(SKEL_1(hpc))]) #struttura delle travi

        verts = list([round(x,3),round(y,3), round(z,3)] for [x,y,z] in ukInfo[0])
        verts = get_new_verts(verts) #rimuovo vertici interni che non servono per la copertura

        #rimuovo piani interni che non servono per la copertura
        cells = remove_base_cells(verts,ukInfo[1])

        faces = STRUCT([OFFSET([0.2,0.2,0.2]),(MKPOL([verts, cells, None]))])

        return STRUCT([T(3)(10),beams,T([3])([0.2]),COLOR(RED)(faces)])


#FINE FUNZIONI AUSILIARIE ----------------------------------------------------------------#

    startRoof = (MKPOL([verts,cells,None])) #tetto di partenza
    return create_roof(startRoof)

### Esempio prima tipologia: Mansard Roof

In [14]:
#TETTO DI PARTENZA---------------------------------------------------------------------------#
verts = [[7.,5.,5.],[7.,-5.,5.],[-7.,5.,5.],[-7.,-5.,5.], #rect interno
          [14.,10.,0],[14.,-10.,0],[-14.,10.,0],[-14.,-10.,0], #rect esterno
          [0,0,8.], #punta centrale
          [0,5.,5.],[0,10.,5.],[0,16.,5.],#punta prolungamento punti alti
          [-7.,10.,0],[7.,10.,0],[-7.,16.,0],[7.,16.,0], #base prolungamento
          [7.,5.,0],[7.,-5.,0],[-7.,5.,0],[-7.,-5.,0], #punti con z=0 per travi basse
          [0,0,5.], #punta centrale media altezza
          [0,10.,0], [0,16.,0] #punta prolungamento punti base
        ]   

cells = [[1,2,5,6,17,18],[1,3,5,7,17,19],[3,4,7,8,19,20],[2,4,6,8,18,20],
         [1,2,3,4,9,17,18,19,20]
        ] 

VIEW(ggpl_mansard_intersecting_roof_builder2(verts,cells))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000001BEEC390> >

<img src = './caso2tipo1.PNG'>

### Esempio secondo Tipologia: Mansard + Intersect Roof

In [16]:
#TETTO DI PARTENZA---------------------------------------------------------------------------#
verts = [[7.,5.,5.],[7.,-5.,5.],[-7.,5.,5.],[-7.,-5.,5.], #rect interno
          [14.,10.,0],[14.,-10.,0],[-14.,10.,0],[-14.,-10.,0], #rect esterno
          [0,0,8.], #punta centrale
          [0,5.,5.],[0,10.,5.],[0,16.,5.],#punta prolungamento punti alti
          [-7.,10.,0],[7.,10.,0],[-7.,16.,0],[7.,16.,0], #base prolungamento
          [7.,5.,0],[7.,-5.,0],[-7.,5.,0],[-7.,-5.,0], #punti con z=0 per travi basse
          [0,0,5.], #punta centrale media altezza
          [0,10.,0], [0,16.,0] #punta prolungamento punti base
        ]   

cells = [[1,2,5,6,17,18],[1,3,5,7,17,19],[3,4,7,8,19,20],[2,4,6,8,18,20],
         [1,2,3,4,9,17,18,19,20],
         [10,13,11,22],[10,14,11,22],
         [13,15,12,11,22,23],[14,16,12,11,22,23]
        ] 

VIEW(ggpl_mansard_intersecting_roof_builder2(verts,cells))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000001BEEC840> >

<img src = './caso2tipo2.PNG'>